# **Introduction to DataFrames**


![](http://spark.apache.org/images/spark-logo.png)


## Objectives


A DataFrame is two-dimensional. Columns can be of different data types. DataFrames accept many data inputs including series and other DataFrames. You can pass indexes (row labels) and columns (column labels). Indexes can be numbers, dates, or strings/tuples.

After completing this lab you will be able to:


* Load a data file into a DataFrame
* View the data schema of a DataFrame
* Perform basic data manipulation
* Aggregate data in a DataFrame


----


## Setup


For this lab, we are going to be using Python and Spark (PySpark). These libraries should be installed in your lab environment or in SN Labs.


Pandas is a popular data science package for Python. In this lab, we use Pandas to load a CSV file from disc to a pandas dataframe in memory. PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context.


In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark
!pip install pandas

In [1]:
import findspark
findspark.init()

In [2]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

##  Spark session


In this exercise, you will create and initialize the Spark session needed to load the dataframes and operate on it


####  Creating the spark session and context


In [3]:

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/04/12 19:10:08 WARN Utils: Your hostname, Mehran resolves to a loopback address: 127.0.0.1; using 192.168.1.106 instead (on interface wlan0)
24/04/12 19:10:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/12 19:10:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


This will give an output similar to:

```
23/10/17 08:29:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
```


####  Initialize Spark session
To work with dataframes we just need to verify that the spark session instance has been created.


In [4]:
spark

##  Load the data and Spark dataframe


In this section, you will first read the CSV file into a Pandas DataFrame and then read it into a Spark DataFrame.
Pandas is a library used for data manipulation and analysis. Pandas offers data structures and operations for creating and manipulating Data Series and DataFrame objects. Data can be imported from various data sources, e.g., Numpy arrays, Python dictionaries, and CSV files. Pandas allows you to manipulate, organize and display the data.
To create a Spark DataFrame we load an external DataFrame, called autompg. This DataFrame includes 398 observations on 10 variables:


| colIndex | colName | units/description |
| :---: | :--- | :--- |
|[1] | mpg |The fuel economy of the car in terms of miles travelled per gallon of gasoline   |
|[2] | cylinders | The number of cylinders in the car's engine   |
|[3] | displacement | The volume of air displaced by all the pistons of a piston engine   |  
|[4] | horsepower  | Horsepower is a measure of power the engine produces    |
|[6] | weight | The total weight of the car   |
|[7] | acceleration | The time in seconds it takes for the car to reach 60 miles per hour |
|[8] | model year | The year (in the 20th century) the car model was released. For example 80 means the car was released in 1980. 
|[9] | origin | The region where the car was manufactured. 1 - USA. 2 - Europe. 3 - Japan 
|[10] | car name | The name of the car model.

####  Loading data into a Pandas DataFrame


In [8]:
# Read the file using `read_csv` function in pandas
autompg = pd.read_csv('Datasets/auto-mpg.csv')

In [9]:
# Preview a few records
autompg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


####  Loading data into a Spark DataFrame


In [10]:
# We use the `createDataFrame` function to load the data into a spark dataframe
sdf = spark.createDataFrame(autompg)

In [11]:
# Let us look at the schema of the loaded spark dataframe
sdf.printSchema()

root
 |-- mpg: double (nullable = true)
 |-- cylinders: long (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- weight: long (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- model year: long (nullable = true)
 |-- origin: long (nullable = true)
 |-- car name: string (nullable = true)



##  Basic data analysis and manipulation

In this section, we perform basic data analysis and manipulation. We start with previewing the data and then applying some filtering and columwise operations.


####  Displays the content of the DataFrame 

We use the `show()` method for this. Here we preview the first 5 records. Compare it to a similar `head()` function in Pandas.


In [12]:
sdf.show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|         ford torino|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



We use the `select()` function to select a particular column of data. Here we show the `mpg` column.


In [13]:
sdf.select('mpg').show(5)

+----+
| mpg|
+----+
|18.0|
|15.0|
|18.0|
|16.0|
|17.0|
+----+
only showing top 5 rows



#### Filtering and Columnar operations

Filtering and Column operations are important to select relevant data and apply useful transformations. 


We first filter to only retain rows with mpg > 18. We use the `filter()` function for this. 


In [14]:
sdf.filter(sdf['mpg'] < 18).show(5)

+----+---------+------------+----------+------+------------+----------+------+-----------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|         car name|
+----+---------+------------+----------+------+------------+----------+------+-----------------+
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|buick skylark 320|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|    amc rebel sst|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|      ford torino|
|15.0|        8|       429.0|       198|  4341|        10.0|        70|     1| ford galaxie 500|
|14.0|        8|       454.0|       220|  4354|         9.0|        70|     1| chevrolet impala|
+----+---------+------------+----------+------+------------+----------+------+-----------------+
only showing top 5 rows



Operating on Columns

Spark also provides a number of functions that can be directly applied to columns for data processing and aggregation. The example below shows the use of basic arithmetic functions to convert the weight values from `lb` to `metric ton`.
We create a new column called `wtTon` that has the weight from the `wt` column converted to metric tons. 


In [15]:
sdf.withColumn('wtTon', sdf['weight'] * 0.45).show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|             wtTon|
+----+---------+------------+----------+------+------------+----------+------+--------------------+------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|            1576.8|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|1661.8500000000001|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|            1546.2|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|1544.8500000000001|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|         ford torino|           1552.05|
+----+---------+------------+----------+------+-

####  Rename the existing column name "horsepower" to "hp" and assign the new result DataFrame to a variable, "sdf_new". 

The function "withColumnRenamed()" renames the existing column names.  


In [16]:
sdf_new = sdf.withColumnRenamed("vs", "versus")

The execution of the above function doesn’t modify the original DataFrame "sdf"; instead, a new DataFrame "sdf_new" is created with the renamed column. 


####  Filter the records based on the condition 

The function "where()" filters the Dataframe rows based on the given condition. It returns a new DataFrame containing the rows that satisfy the given condition. 


In [17]:
sdf.where(sdf['mpg'] < 18).show(3) 

+----+---------+------------+----------+------+------------+----------+------+-----------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|         car name|
+----+---------+------------+----------+------+------------+----------+------+-----------------+
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|buick skylark 320|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|    amc rebel sst|
|17.0|        8|       302.0|       140|  3449|        10.5|        70|     1|      ford torino|
+----+---------+------------+----------+------+------------+----------+------+-----------------+
only showing top 3 rows



>Note: Both filter() and where() functions are used for the same purpose. 


#### Combining DataFrames based on a specific condition. 

The function "join()"combines the DataFrames based on a specific condition. 

See the below examples.


In [20]:
# define sample DataFrame 1 

data = [("A101", "Mehran"), ("A102", "Sina"), ("A103", "Sobhan")] 

columns = ["emp_id", "emp_name"] 

dataframe_1 = spark.createDataFrame(data, columns) 

In [21]:
# define sample DataFrame 2 

data = [("A101", 1000), ("A102", 2000), ("A103", 3000)]

columns = ["emp_id", "salary"]

dataframe_2 = spark.createDataFrame(data, columns)

In [22]:
# create a new DataFrame, "combined_df" by performing an inner join

combined_df = dataframe_1.join(dataframe_2, on="emp_id", how="inner")

In [23]:
combined_df.show()

+------+--------+------+
|emp_id|emp_name|salary|
+------+--------+------+
|  A101|  Mehran|  1000|
|  A102|    Sina|  2000|
|  A103|  Sobhan|  3000|
+------+--------+------+



####  Filling the missing values 

"fillna()" or "fill()" function fill the missing values with a specified value. 


In [24]:
# define sample DataFrame 1

data = [("A101", 1000), ("A102", 2000), ("A103",None)]

columns = ["emp_id", "salary"]

dataframe_1 = spark.createDataFrame(data, columns)

Note that the third record of the DataFrame "dataframe_1", the column “salary”, contains null("na") value. It can be filled with a value by using the function "fillna()". 


In [25]:
# fill missing salary value with a specified value 

filled_df = dataframe_1.fillna({"salary": 3000}) 
filled_df.head(3)

[Row(emp_id='A101', salary=1000),
 Row(emp_id='A102', salary=2000),
 Row(emp_id='A103', salary=3000)]

## Grouping and Aggregation

Spark DataFrames support a number of commonly used functions to aggregate data after grouping. In this example we compute the average weight of cars by their cylinders as shown below.


In [26]:
sdf.groupby(['cylinders'])\
.agg({"weight": "AVG"})\
.show(5)

+---------+------------------+
|cylinders|       avg(weight)|
+---------+------------------+
|        6|3198.2261904761904|
|        8| 4114.718446601942|
|        4| 2308.127450980392|
|        3|            2398.5|
|        5|3103.3333333333335|
+---------+------------------+



We can also sort the output from the aggregation to get the most common cars.


In [27]:
car_counts = sdf.groupby(['cylinders'])\
.agg({"weight": "count"})\
.sort("count(weight)", ascending=False)\
.show(5)


+---------+-------------+
|cylinders|count(weight)|
+---------+-------------+
|        4|          204|
|        8|          103|
|        6|           84|
|        3|            4|
|        5|            3|
+---------+-------------+



## Authors


[Mehran Morabbi Pazoki](https://www.linkedin.com/in/mehran-pazoki-6a3372175/)

